# Meeting Log Summary

The aim of this notebook is to generate three CSV files that will be used to display the Dashboard on Google Data Studio:
- meeting_group_march_sept.csv (Meetings duration)
- group_by_meetingCode.csv (Participants punctuality)
- mentors.csv (Mentors geolocation) <br>

Link to the Dashboard : https://datastudio.google.com/reporting/1df8411a-0c41-41ca-986b-0ba98b119932/page/p_bw2m1tmnpc
 

## Data Input

Two files are needed for the creation of the Dashboard:
- calendar.csv (Run code below which need credentials file *service-acc.json*)
- meetlogs.csv (Already provided)

NB: Make sure the files (notebook + files) are located in the same folder.

In [1]:
from google.oauth2 import service_account
from googleapiclient.discovery import build
import pandas as pd
import os
from datetime import timedelta

In [3]:
class google_apis:
    __webdev_cred = ''
    __mentor_cred = ''

    def __init__(self):
        scopes = [
            'https://www.googleapis.com/auth/calendar',
            'https://www.googleapis.com/auth/gmail.compose',
            'https://www.googleapis.com/auth/admin.directory.user',
            'https://www.googleapis.com/auth/admin.directory.group',
        ]
        SERVICE_ACCOUNT_FILE = os.path.join("", "service-acc.json")
        credentials = service_account.Credentials.from_service_account_file(
            'service-acc.json', scopes=scopes)
        self.__webdev_cred = credentials.with_subject(
            'webdevelopment@villagebookbuilders.org')
        self.__mentor_cred = credentials.with_subject(
            'mentor@villagebookbuilders.org')

    def calendar_data_arr(self):
        data_arr = [['Associated Calendar', 'Meet Link', 'Participants', 'Start Time', 'End Time', 'ID']]
        calendar_service = build('calendar', 'v3', credentials=self.__mentor_cred)
        ids = calendar_service.calendarList().list().execute()
        for calendar_list_entry in ids['items']:
            list = calendar_service.events().list(calendarId=calendar_list_entry['id']).execute()
            flag = True
            for event in list['items']:
                if 'hangoutLink' in event and 'attendees' in event:
                    data_arr.append([calendar_list_entry['summary'],
                                    event['hangoutLink'], event['attendees'], event['start'], event['end'], event['id']])
        df = pd.DataFrame(data_arr)
        df.to_csv('calendar.csv')

api_obj = google_apis()
api_obj.calendar_data_arr()
print('File calendar.csv created !')

File calendar.csv created !


In [2]:
# Read the two CSV files
data_meetlogs = pd.read_excel('meetlogs.xlsx')
data_calendar = pd.read_csv('calendar.csv', header=1)

## First Page : Time spent in meetings

In [19]:
# # Select the columns of interest 
data = data_meetlogs[['Date', 'Meeting Code', 'Participant Identifier', 'Duration', 'Participant Name']].copy()

# Modify the Date column format
data['Date'] = pd.to_datetime(data['Date']).dt.strftime('%m/%d/%Y')

#Sum durations to even time
data = data.groupby(['Date', 'Meeting Code', 'Participant Identifier', 'Participant Name']).sum().reset_index()

# Add two new columns
username = 'mphightech|hightech1|hightech2|hightech3|hightech4|hightech.announcements|hightech.collaboration|mpchebosi|\
chebosi1|chebosi2|chebosi3|chebosi.announcements|chebosi.collaboration|mpahero|ahero1|ahero2|ahero3|ahero4|\
ahero5|ahero6|ahero.announcements|ahero.collaboration|mukono1|mukono2|mukono.announcements|mukono.collaboration|\
sanlazaro|sanlazaro1|sanlazaro2|sanlazaro3|sanlazaro4|sanlazaro5|sanlazaro6|sanlazaro7|sakubunase2|sakubunase3|\
mpadeiso|adeiso1|adeiso2|adeiso3|adeiso4|adeiso.announcements@villagebookbuilders.org|adeiso.collaboration@villagebookbuilders.org|\
mpkadzakalowa|kadzakalowa1|kadzakalowa2|kadzakalowa3|kadzakalowa4|kadzakalowa5|kadzakalowa6|kadzakalowa7|kadzakalowa8|\
kadzakalowa9|kadzakalowa10|mpcharitycentre|charitycentre1|charitycentre2|charitycentre3|charitycentre4|charitycentre5|\
charitycentre6|charitycentre7|charitycentre8|charitycentreannouncements|charitycentre.collab|mpjollyland|jollyland1|\
jollyland2|jollyland3|jollyland4|jollyland.annoucements|jollyland.collaboration|mpmaragoli|maragoli1|maragoli2|maragoli3|\
maragoli4|maragoli.announcements|maragoli.collaboration|mpyocreek|yocreek1|yocreek2|yocreek3|yocreek4|yocreek5|yocreek6|\
yocreek7|yocreek8|yocreek9|yocreek10|mpvima|vima1|vima2|vima3|vima4|mpbiwi|biwi1|biwi2|biwi3|biwiannouncements@villagebookbuilders.org|\
biwi.collaboration@villagebookbuilders.org'

sub = "villagementors.org"
data['is_mentee']= data["Participant Identifier"].str.contains(username)
data['is_mentor']= data["Participant Identifier"].str.contains(sub)
#data['is_mentor'] = data['is_mentee'].apply(lambda x: not x)

# Attribute the role of each participant
def identify_type(is_mentee,is_mentor):
    if (is_mentee == True):
        return "Mentee"
    if (is_mentor == True):
        return "Mentor"
    else:
        return "Staff"

data['Identify_type'] = data.apply(lambda x: identify_type(x.is_mentee,x.is_mentor), axis=1)

# Convert new dataframe to csv file
data.to_csv('meeting_group_march_sept.csv')
print("File meeting_group_march_sept.csv created !")

File meeting_group_march_sept.csv created !


## Second Page : Participants punctuality 

In [18]:
# Preprocess calendar.csv
data_c = data_calendar[['Meet Link', 'Start Time', 'End Time']].copy()
data_c['Meeting Code'] = data_c['Meet Link'].str.strip().str[-12:].str.split('-').str.join("").str.upper()
data_c['startDateTime'] = data_c['Start Time'].apply(lambda x: eval(x).get('dateTime'))
data_c['endDateTime'] = data_c['End Time'].apply(lambda x: eval(x).get('dateTime'))
data_c['Date_format'] = data_c['startDateTime'].str.strip().str[:10]
data_c = data_c[['Date_format', 'Meeting Code', 'startDateTime', 'endDateTime']]

# Preprocess meetlogs.csv
data_m = data_meetlogs[['Date', 'Meeting Code', 'Participant Identifier', 'Duration', 'Participant Name']].copy()
data_m['Date_format'] = pd.to_datetime(data_m['Date']).dt.strftime('%Y-%m-%d')

# Join the two csv files
data_merged = pd.merge(data_m, data_c, on=['Date_format','Meeting Code'])

#Process data_merged
data_merged['startDateTime'] = pd.to_datetime(data_merged['startDateTime'], utc=True)[0]
data_merged['endDateTime'] = pd.to_datetime(data_merged['endDateTime'], utc=True)[0]
data_merged['UTC'] = pd.to_datetime(data_merged['Date'], utc=True)

data_merged['is_mentee']= data_merged["Participant Identifier"].str.contains(username)
data_merged['is_mentor']= data_merged["Participant Identifier"].str.contains(sub)
#data_merged['is_mentor'] = data_merged['is_mentee'].apply(lambda x: not x)

# Attribute the role of each participant
def identify_type(is_mentee,is_mentor):
    if (is_mentee == True):
        return "Mentee"
    if (is_mentor == True):
        return "Mentor"
    else:
        return "Staff"

data_merged['Identify_type'] = data_merged.apply(lambda x: identify_type(x.is_mentee,x.is_mentor), axis=1)

def get_real_end(meeting_code, date_format, participant_identifier, participant_name):
    return data_merged[(data_merged['Meeting Code'] == meeting_code) &
                (data_merged['Date_format'] == date_format) &
                (data_merged['Participant Identifier'] == participant_identifier) &
                (data_merged['Participant Name'] == participant_name)]['UTC'].max()

#Group df to sum duration
df = data_merged.groupby(['Date_format', 'Meeting Code', 'Participant Identifier',
       'Participant Name', 'startDateTime', 'endDateTime',
       'Identify_type']).sum().reset_index()

#Uniformize dates
df['UTC'] = df.apply(lambda x: get_real_end(x['Meeting Code'], x['Date_format'], x['Participant Identifier'], x['Participant Name']), axis = 1)

#Middle point dates to calculate difference beetwen calendar and meetlog
df['durationUTC'] = df.UTC - df.Duration * timedelta(seconds=1)
df['meetStartUTC'] = df['UTC'].dt.floor('H') #Based on UTC Column
df['meetEndUTC'] = df['UTC'].dt.ceil('H')
df['startDateTime'] = pd.to_datetime(df['startDateTime'], utc=True)
df['endDateTime'] = pd.to_datetime(df['endDateTime'], utc=True)
df['meetStartDuration'] = (df['durationUTC']-timedelta(minutes = 15)).dt.round('H')
df['meetEndDuration'] = df.durationUTC + (df.endDateTime - df.startDateTime)
df['deltaStart'] = df['startDateTime'] - df['meetStartDuration']
df['deltaEnd'] = df['endDateTime'] - df['meetEndDuration']

#Clean dates
df['cleanStartCalendar'] = df.startDateTime - df.deltaStart
df['cleanEndCalendar'] = df.endDateTime - df.deltaStart
df['meetingEnd'] = df.UTC
df['meetingStart'] = df.durationUTC
df['meetingDate'] = df['cleanStartCalendar'].dt.date

#Only clean information
clean_df = df[['Meeting Code','Identify_type','meetingDate','meetingStart','meetingEnd','cleanStartCalendar','cleanEndCalendar', 'Duration']].copy()

#Calculate punctuality
clean_df['punctuality'] = (clean_df['meetingStart'] - clean_df['cleanStartCalendar']).dt.total_seconds()

#Save
clean_df.to_csv('group_by_meetingCode.csv')
print('File group_by_meetingCode.csv saved!')

group_by_meetingCode.csv saved!


## Third Page : Mentors geolocation 

In [9]:
# Select the columns of interest 
data = data_meetlogs[['Participant Identifier', 'Participant Name', 'Organizer Email', 'City', 'Country']]

# Select only rows where the Organizer Email is 'mentor@villagebookbuilders.org' (the remainings are supposed not to be related to the mentoring program) and drop the column
data_mp = data.loc[data['Organizer Email'] == 'mentor@villagebookbuilders.org']
data_mp = data_mp.drop(columns=['Organizer Email'])

# Find all the mentees (whose either Identifier starts with 'mp' or Name with 'Computer') et discard them from the dataframe
data_mentors = data_mp.loc[~(data['Participant Identifier'].str.startswith('mp', na=False) | data['Participant Name'].str.startswith('Computer', na=False))]

# Drop all duplicates from the Participant Identifier
data_mentors = data_mentors.drop_duplicates(subset=['Participant Identifier'])

# Convert filtered dataframe to csv file
data_mentors.to_csv('mentors.csv', index=False)
print("File mentors.csv created !")

File mentors.csv created !
